In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [2]:
spark = SparkSession.builder.master("local[*]").appName("Demo").getOrCreate()
spark

23/04/04 22:08:47 WARN Utils: Your hostname, ZSCHN01LP0253L resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
23/04/04 22:08:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 22:08:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/04 22:08:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/04 22:08:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/04 22:08:48 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/04/04 22:08:48 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/04/04 22:08:48 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/04/04 22:08:48 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [3]:
empDf = spark.read.option("header",True).option("infrechema",True).csv("../../Spark-main/employees.csv")
empDf.show(1)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 1 row



In [4]:
deptDf = spark.read.option("header",True).option("infrechema",True).csv("../../Spark-main/departments.csv")
deptDf.show(1)

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
+-------------+---------------+----------+-----------+
only showing top 1 row



In [5]:
from pyspark.sql.functions import *

In [6]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)
#The Value is in Bytes

In [7]:
resultDf=empDf.join(broadcast(deptDf), empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "inner").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME)
resultDf.show(1)

+-----------+-------------+---------------+
|EMPLOYEE_ID|DEPARTMENT_ID|DEPARTMENT_NAME|
+-----------+-------------+---------------+
|        198|           50|       Shipping|
+-----------+-------------+---------------+
only showing top 1 row



In [8]:
empDf.rdd.getNumPartitions()

1

In [9]:
deptDf.rdd.getNumPartitions()

1

In [11]:
resultDf.rdd.getNumPartitions()

1

In [18]:
resultDf.repartition(10)

DataFrame[EMPLOYEE_ID: string, DEPARTMENT_ID: string, DEPARTMENT_NAME: string]

In [19]:
resultDf.rdd.getNumPartitions()

1

In [20]:
newDf = resultDf.repartition(10)
newDf.rdd.getNumPartitions()

10

In [21]:
df1 = newDf.repartition(2)

In [22]:
df1.rdd.getNumPartitions()

2

In [23]:
newDf.rdd.getNumPartitions()

10

### coalesce()

In [24]:
df2 = newDf.coalesce(20)
df2.rdd.getNumPartitions()

10

In [26]:
df3 = newDf.coalesce(5)
df3.rdd.getNumPartitions()

5

In [27]:
resultDf.coalesce(1).write.mode("overwrite").option("header",True).format("csv").save("/tmp/result")